# **ConvLSTM for cwnd 3 Algos**

**Importing necessary packages, modules**


In [ ]:
# convlstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from matplotlib import pyplot

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import io

**Getting dataset : x_array, y_array (from file)**

In [ ]:
from google.colab import files
uploaded = files.upload()

df_x = pd.read_csv(io.BytesIO(uploaded['x_dataset_full.csv']),header=None)
data_x = df_x.values
df_y = pd.read_csv(io.BytesIO(uploaded['y_dataset_full.csv']),header=None)
data_y = df_y.values

X = np.array(data_x).reshape(3000,128,1)
Y = np.array(data_y)
# Y1 = np.array(data_y1)
# encoder = OneHotEncoder(sparse=False)
Y = pd.get_dummies(df_y).values
print(X)
print(Y)

Saving x_dataset_full.csv to x_dataset_full (1).csv
Saving y_dataset_full.csv to y_dataset_full (1).csv
[[[0.1       ]
  [0.11750547]
  [0.13369803]
  ...
  [0.88949672]
  [0.89387308]
  [0.89737418]]

 [[0.1       ]
  [0.12830882]
  [0.15082721]
  ...
  [0.89191177]
  [0.89494485]
  [0.89797794]]

 [[0.1       ]
  [0.12945959]
  [0.15289335]
  ...
  [0.89081779]
  [0.89426112]
  [0.89770445]]

 ...

 [[0.1       ]
  [0.1       ]
  [0.11695322]
  ...
  [0.9       ]
  [0.9       ]
  [0.9       ]]

 [[0.1       ]
  [0.1       ]
  [0.11287356]
  ...
  [0.89126437]
  [0.89126437]
  [0.9       ]]

 [[0.1       ]
  [0.1       ]
  [0.11591198]
  ...
  [0.9       ]
  [0.9       ]
  [0.9       ]]]
[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


**Train Test Split**

In [ ]:
# # split a dataset into train and test sets
from sklearn.model_selection import train_test_split

# split into train test sets
trainX, testX, trainy, testy = train_test_split(X, Y, test_size=0.33, random_state=64)

print(trainX.shape, testX.shape, trainy.shape, testy.shape)

(2010, 128, 1) (990, 128, 1) (2010, 3) (990, 3)


**ConvLSTM Model**

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 1, 100, 32
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	# model.add(Dropout(0.5))
	model.add(Flatten())
	# model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
	return accuracy

**Summarize Results**

In [ ]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=1):
	# # load data
	# trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

**Run**

In [ ]:
# run the experiment
run_experiment()

Epoch 1/100
63/63 [==============================] - 7s 55ms/step - loss: 1.1012 - accuracy: 0.3213
Epoch 2/100
63/63 [==============================] - 3s 54ms/step - loss: 1.0928 - accuracy: 0.3655
Epoch 3/100
63/63 [==============================] - 3s 54ms/step - loss: 0.9838 - accuracy: 0.6084
Epoch 4/100
63/63 [==============================] - 3s 55ms/step - loss: 0.6362 - accuracy: 0.7041
Epoch 5/100
63/63 [==============================] - 3s 55ms/step - loss: 0.5885 - accuracy: 0.7168
Epoch 6/100
63/63 [==============================] - 3s 54ms/step - loss: 0.5543 - accuracy: 0.7211
Epoch 7/100
63/63 [==============================] - 3s 55ms/step - loss: 0.5338 - accuracy: 0.7443
Epoch 8/100
63/63 [==============================] - 3s 55ms/step - loss: 0.5514 - accuracy: 0.7293
Epoch 9/100
63/63 [==============================] - 3s 54ms/step - loss: 0.5715 - accuracy: 0.7179
Epoch 10/100
63/63 [==============================] - 3s 54ms/step - loss: 0.5148 - accuracy: 0.7513